In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR
import holodeck.anisotropy as anis

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



In [ ]:
nn=346
sspath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss51-2023-05-22_uniform_07a_n1000_r100_f40_l2000/'
hdfname = sspath+'sam_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][nn]
hc_bg = ssfile['hc_bg'][nn]
fobs = ssfile['fobs'][:]
# dfobs = ssfile['dfobs'][:]
ssfile.close()

shape=hc_ss.shape
nfreqs = shape[-3]
nreals = shape[-2]
nloudest = shape[-1]
print('nfreqs=%d, nreals=%d, nloudest=%d' % (nfreqs, nreals, nloudest))

In [ ]:
# spread background evenly across pixels

def make_bg_map(hc_bg, nside):
    npix = hp.nside2npix(nside)
    moll_bg = (np.ones((hc_bg.shape + (npix,))) 
            * hc_bg[:,:,np.newaxis]/np.sqrt(npix)) # (frequency, realization, pixel)
    return moll_bg

nsides=np.array([8,16,32, 64, 128])
moll_bg08 = make_bg_map(hc_bg, nside=8)
moll_bg16 = make_bg_map(hc_bg, nside=16)
moll_bg32 = make_bg_map(hc_bg, nside=32)
moll_bg64 = make_bg_map(hc_bg, nside=64)
moll_bg128 = make_bg_map(hc_bg, nside=128)
print(moll_bg08.shape)


In [ ]:
# place single sources at random pixels, for one map

def make_ss_map(hc_ss, nside, pix_ss=None):
    npix = hp.nside2npix(nside)
    if pix_ss is None:
        pix_ss = np.random.randint(0, npix-1, size=hc_ss.size).reshape(hc_ss.shape)
    moll_ss = np.zeros((len(hc_ss), len(hc_ss[0]), npix,))
    for ff in range(hc_ss.shape[-3]):
        for rr in range(hc_ss.shape[-2]):
            for ll in range(hc_ss.shape[-1]):
                moll_ss[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_ss[ff,rr,pix_ss[ff,rr,ll]]**2
                                                         + hc_ss[ff,rr,ll]**2)
    return moll_ss, pix_ss
moll_ss08, pix_ss08 = make_ss_map(hc_ss, 8)
print(f"{moll_ss08.shape=}")
print(f"{pix_ss08.shape=}")

In [ ]:
def convert_pix_ss(pix_ss1, nside1, nside2):
    theta, phi = hp.pix2ang(nside1, pix_ss1)
    print(f"{theta.shape=}, {phi.shape=}")
    pix_ss2 = hp.ang2pix(nside2, theta, phi).reshape(pix_ss1.shape)
    return pix_ss2

pix_ss16 = convert_pix_ss(pix_ss08, 8, 16)
pix_ss32 = convert_pix_ss(pix_ss08, 8, 32)
pix_ss64 = convert_pix_ss(pix_ss08, 8, 64)
pix_ss128 = convert_pix_ss(pix_ss08, 8, 128)

In [ ]:
moll_ss16, temp = make_ss_map(hc_ss, 16, pix_ss16)
moll_ss32, temp = make_ss_map(hc_ss, 32, pix_ss32)
moll_ss64, temp = make_ss_map(hc_ss, 64, pix_ss64)
moll_ss128, temp = make_ss_map(hc_ss, 128, pix_ss128)

In [ ]:
moll_tt08 = np.sqrt(moll_ss08**2 + moll_bg08**2)
moll_tt16 = np.sqrt(moll_ss16**2 + moll_bg16**2)
moll_tt32 = np.sqrt(moll_ss32**2 + moll_bg32**2)
moll_tt64 = np.sqrt(moll_ss64**2 + moll_bg64**2)
moll_tt128 = np.sqrt(moll_ss128**2 + moll_bg128**2)

In [ ]:

rr=0
for ff in range(0,5):
    fig, axs = plot.figax(ncols=3, figsize=(15,5))
    plt.axes(axs[0])
    hp.mollview(moll_tt08[ff,rr], title='f=%.2f yr$^{-1}$, nside=%d'
                % (fobs[ff]*YR,  8), hold=True )
    plt.axes(axs[1])
    hp.mollview(moll_tt16[ff,rr], title='f=%.2f yr$^{-1}$, nside=%d'
                % (fobs[ff]*YR, 16), hold=True )
    plt.axes(axs[2])
    hp.mollview(moll_tt32[ff,rr], title='f=%.2f yr$^{-1}$, nside=%d'
                % (fobs[ff]*YR, 32), hold=True )
    fig.tight_layout()


In [ ]:
lmax=6
Cl_08 = anis.sph_harm_from_map(moll_tt08, lmax=6)
Cl_16 = anis.sph_harm_from_map(moll_tt16, lmax=6)
Cl_32 = anis.sph_harm_from_map(moll_tt32, lmax=6)
Cl_64 = anis.sph_harm_from_map(moll_tt64, lmax=6)
Cl_128 = anis.sph_harm_from_map(moll_tt128, lmax=6)

In [ ]:
Cl_arrs = np.array([Cl_08, Cl_16, Cl_32, Cl_64, Cl_128])
print(Cl_arrs.shape)

In [ ]:
rr = 0
ll=1

xx = np.arange(lmax+1)
colors = cm.rainbow(np.linspace(0,1,len(Cl_arrs)))

for ff in np.array([1,5,10]):
    yy = Cl_arrs
    fig, ax = plot.figax(xlabel='$\ell$', ylabel='$C_\ell$')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, yy[ii,ff,rr,:], color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
    ax.legend()
    ax.set_title('Sample %d, Realization %d' % (nn, rr))
    fig.tight_layout()

In [ ]:
rr = 0
ll=1

xx = fobs*YR
colors = cm.rainbow(np.linspace(0,1,len(Cl_arrs)))

for ll in range(1,7):
    yy = Cl_arrs[...,ll]/Cl_arrs[...,0]
    fig, ax = plot.figax(xlabel='$f$ yr$^{-1}$', ylabel='$C_1/C_0$')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, yy[ii,:,rr], color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
    ax.legend()
    ax.set_title('Sample %d, Realization %d' % (nn, rr))
    fig.tight_layout()

In [ ]:
ll=1

xx = fobs*YR
colors = cm.rainbow(np.linspace(0,1,len(Cl_arrs)))

for ll in range(1,7):
    yy = Cl_arrs[...,ll]/Cl_arrs[...,0]
    fig, ax = plot.figax(xlabel='$f$ yr$^{-1}$', ylabel='$C_1/C_0$')
    for ii, nside in enumerate(nsides):
        ax.plot(xx, np.median(yy[ii], axis=-1), color=colors[ii], label=('$\ell=%d$, $\ell_\mathrm{max}=%d,$ nside=%d' % (ll, lmax, nside)))
        for pp in [50,]:
            percs = pp/2
            percs = [50-percs, 50+percs]
            ax.fill_between(xx, *np.percentile(yy[ii], percs, axis=-1), alpha=0.25, color=colors[ii])

    ax.legend()
    ax.set_title('Sample %d, Realization %d' % (nn, rr))
    fig.tight_layout()